In [1]:

class STree():
    '''Class representing the suffix tree.'''

    def __init__(self, input=[]):
        self.root = _SNode()
        self.root.depth = 0
        self.root.idx = 0
        self.root.parent = self.root
        self.root._add_suffix_link(self.root)

        if not input:
            self.build(input)

    def build(self, x):
        '''Builds the Suffix tree on the given input.
        If the input is of type List of Lists:
        Generalized Suffix Tree is built.

        :param x: List or List of Lists
        '''
        type = self._check_input(x)

        if type == 'st':
            x += next(self._terminalSymbolsGenerator())
            self._build(x)
        if type == 'gst':
            self._build_generalized(x)

    def _check_input(self, input):
        '''Checks the validity of the input.

        In case of an invalid input throws ValueError.
        '''
        if isinstance(input, list):
            return 'st'
        elif isinstance(input, list):
            if all(isinstance(item, list) for item in input):
                return 'gst'

        raise ValueError('Input argument should be of type List or a list of lists')

    def _build(self, x):
        '''Builds a Suffix tree.'''
        self.word = x
        self._build_McCreight(x)

    def _build_McCreight(self, x):
        '''Builds a Suffix tree using McCreight O(n) algorithm.

        Algorithm based on:
        McCreight, Edward M. 'A space-economical suffix tree construction algorithm.' - ACM, 1976.
        Implementation based on:
        UH CS - 58093 String Processing Algorithms Lecture Notes
        '''
        u = self.root
        d = 0
        for i in range(len(x)):
            while u.depth == d and u._has_transition(x[d + i]):
                u = u._get_transition_link(x[d + i])
                d = d + 1
                while d < u.depth and x[u.idx + d] == x[i + d]:
                    d = d + 1
            if d < u.depth:
                u = self._create_node(x, u, d)
            self._create_leaf(x, i, u, d)
            if not u._get_suffix_link():
                self._compute_slink(x, u)
            u = u._get_suffix_link()
            d = d - 1
            if d < 0:
                d = 0

    def _create_node(self, x, u, d):
        i = u.idx
        p = u.parent
        v = _SNode(idx=i, depth=d)
        v._add_transition_link(u, x[i + d])
        u.parent = v
        p._add_transition_link(v, x[i + p.depth])
        v.parent = p
        return v

    def _create_leaf(self, x, i, u, d):
        w = _SNode()
        w.idx = i
        w.depth = len(x) - i
        u._add_transition_link(w, x[i + d])
        w.parent = u
        return w

    def _compute_slink(self, x, u):
        d = u.depth
        v = u.parent._get_suffix_link()
        while v.depth < d - 1:
            v = v._get_transition_link(x[u.idx + v.depth + 1])
        if v.depth > d - 1:
            v = self._create_node(x, v, d - 1)
        u._add_suffix_link(v)

    def _build_Ukkonen(self, x):
        '''Builds a Suffix tree using Ukkonen's online O(n) algorithm.

        Algorithm based on:
        Ukkonen, Esko. 'On-line construction of suffix trees.' - Algorithmica, 1995.
        '''
        # TODO.
        raise NotImplementedError()

    def _build_generalized(self, xs):
        '''Builds a Generalized Suffix Tree (GST) from the list of lists of ints provided.
        '''
        terminal_gen = self._terminalSymbolsGenerator()

        _xs = ''.join([x + next(terminal_gen) for x in xs])
        self.word = _xs
        self._generalized_word_starts(xs)
        self._build(_xs)
        self.root._traverse(self._label_generalized)

    def _label_generalized(self, node):
        '''Helper method that labels the nodes of GST with indexes of lists
        found in their descendants.
        '''
        if node.is_leaf():
            x = {self._get_word_start_index(node.idx)}
        else:
            x = {n for ns in node.transition_links.values() for n in ns.generalized_idxs}
        node.generalized_idxs = x

    def _get_word_start_index(self, idx):
        '''Helper method that returns the index of the list based on node's
        starting index'''
        i = 0
        for _idx in self.word_starts[1:]:
            if idx < _idx:
                return i
            else:
                i += 1
        return i

    def lcs(self, listIdxs=-1):
        '''Returns the Largest Common Sublist of Lists provided in listIdxs.
        If listIdxs is not provided, the LCS of all lists is returned.

        ::param listIdxs: Optional: List of indexes of lists.
        '''
        if listIdxs == -1 or not isinstance(listIdxs, list):
            listIdxs = set(range(len(self.word_starts)))
        else:
            listIdxs = set(listIdxs)

        deepestNode = self._find_lcs(self.root, listIdxs)
        start = deepestNode.idx
        end = deepestNode.idx + deepestNode.depth
        return self.word[start:end]

    def _find_lcs(self, node, listIdxs):
        '''Helper method that finds LCS by traversing the labeled GSD.'''
        nodes = [self._find_lcs(n, listIdxs)
                 for n in node.transition_links.values()
                 if n.generalized_idxs.issuperset(listIdxs)]

        if nodes == []:
            return node

        deepestNode = max(nodes, key=lambda n: n.depth)
        return deepestNode

    def _generalized_word_starts(self, xs):
        '''Helper method returns the starting indexes of lists in GST'''
        self.word_starts = []
        i = 0
        for n in range(len(xs)):
            self.word_starts.append(i)
            i += len(xs[n]) + 1

    def find(self, y):
        '''Returns starting position of the sublist y in the list used for
        building the Suffix tree.

        :param y: List
        :return: Index of the starting position of list y in the list used for building the Suffix tree
                 -1 if y is not a sublist.
        '''
        node = self.root
        while True:
            edge = self._edgeLabel(node, node.parent)
            if edge.startswith(y):
                return node.idx

            i = 0
            while (i < len(edge) and edge[i] == y[0]):
                y = y[1:]
                i += 1

            if i != 0:
                if i == len(edge) and y != '':
                    pass
                else:
                    return -1

            node = node._get_transition_link(y[0])
            if not node:
                return -1

    def find_all(self, y):
        node = self.root
        while True:
            edge = self._edgeLabel(node, node.parent)
            if edge.startswith(y):
                break

            i = 0
            while (i < len(edge) and edge[i] == y[0]):
                y = y[1:]
                i += 1

            if i != 0:
                if i == len(edge) and y != '':
                    pass
                else:
                    return {}

            node = node._get_transition_link(y[0])
            if not node:
                return {}

        leaves = node._get_leaves()
        return {n.idx for n in leaves}

    def _edgeLabel(self, node, parent):
        '''Helper method, returns the edge label between a node and it's parent'''
        return self.word[node.idx + parent.depth: node.idx + node.depth]

    def _terminalSymbolsGenerator(self):
        '''Generator of unique terminal symbols used for building the Generalized Suffix Tree.
        Unicode Private Use Area U+E000..U+F8FF is used to ensure that terminal symbols
        are not part of the input string.
        '''
        UPPAs = list(list(range(0xE000, 0xF8FF+1)) +
                     list(range(0xF0000, 0xFFFFD+1)) + list(range(0x100000, 0x10FFFD+1)))
        for i in UPPAs:
            yield (chr(i))

        raise ValueError('Too many input strings.')

In [2]:

class _SNode():
    __slots__ = ['_suffix_link', 'transition_links', 'idx', 'depth', 'parent', 'generalized_idxs']

    '''Class representing a Node in the Suffix tree.'''

    def __init__(self, idx=-1, parentNode=None, depth=-1):
        # Links
        self._suffix_link = None
        self.transition_links = {}
        # Properties
        self.idx = idx
        self.depth = depth
        self.parent = parentNode
        self.generalized_idxs = {}

    def __str__(self):
        return ('SNode: idx:' + str(self.idx) + ' depth:' + str(self.depth) +
                ' transitons:' + str(list(self.transition_links.keys())))

    def _add_suffix_link(self, snode):
        self._suffix_link = snode

    def _get_suffix_link(self):
        if self._suffix_link is not None:
            return self._suffix_link
        else:
            return False

    def _get_transition_link(self, suffix):
        return False if suffix not in self.transition_links else self.transition_links[suffix]

    def _add_transition_link(self, snode, suffix):
        self.transition_links[suffix] = snode

    def _has_transition(self, suffix):
        return suffix in self.transition_links

    def is_leaf(self):
        return len(self.transition_links) == 0

    def _traverse(self, f):
        for node in self.transition_links.values():
            node._traverse(f)
        f(self)

    def _get_leaves(self):
        # Python <3.6 dicts don't perserve insertion order (and even after, we
        # shouldn't rely on dicts perserving the order) therefore these can be
        # out-of-order, so we return a set of leaves.
        if self.is_leaf():
            return {self}
        else:
            return {x for n in self.transition_links.values() for x in n._get_leaves()}

In [4]:

navigable_parent_ids_list = [3104, 3119, 3462, 3459, 14268, 14270, 14258, 14266, 14254, 14249, 3117, 2564,
                             14246, 2562, 2801, 2901, 3116, 832, 1758, 15616, 309, 13306, 1640, 2077, 15585,
                             2080, 15594, 505, 10133, 7507, 4625, 8092, 4549, 6271, 8648, 4531, 4302, 2092,
                             14981, 271, 2033, 12934, 1894, 12921, 1597, 11916, 1206, 11921, 1818, 8761,
                             10274, 10272, 13217, 1638, 685, 9901, 5124, 8903, 1613, 13959, 527, 1022, 15958,
                             1405, 12284, 1017, 15960, 6678, 6898, 6940, 6899, 7398, 6673, 6706, 6714, 6963,
                             10762, 10835, 878, 1545, 880, 570, 984, 1268, 1502, 881, 1031, 1244, 1291, 1287,
                             1409, 1764, 1769, 1770, 1771, 1773, 1245, 1052, 1379, 1395, 2109, 1382, 1326,
                             1003, 1161, 1311, 1512, 1917, 2138, 519, 1068, 759, 625, 817, 1780, 1853, 1130,
                             2300, 1292, 1380, 583, 584, 1868, 1844, 538, 1247, 1729, 482, 641, 780, 983,
                             989, 1036, 1522, 572, 624, 1258, 620, 1039, 1646, 1415, 860, 1152, 1355, 1482,
                             1248, 466, 567, 586, 1381, 1728, 627, 650, 1129, 1413, 2040, 1523, 1501, 1510,
                             1234, 1041, 2108, 1000, 1324, 1376, 521, 982, 997, 1072, 1768, 1412, 1766, 1394,
                             1725, 2069, 1783, 1329, 2110, 1222, 1318, 446, 599, 9924, 15668, 2408, 16254,
                             11102, 11101, 3817, 8957, 5374, 8143, 7983, 7843, 10909, 120, 1349, 15168, 1076,
                             13968, 15982, 15699, 1657, 326, 14376, 2498, 16192, 14958, 2075, 1, 15714, 1215,
                             2018, 2413, 537, 1787, 12925, 820, 13246, 1677, 332, 144, 470, 16022, 2055,
                             14962, 987, 609, 14959, 836, 14967, 14363, 14284, 14966, 1109, 12423, 12537,
                             12200, 564, 11940, 12982, 5102, 10717, 4288, 3975, 3951, 8515, 12434, 14922,
                             1996, 1978, 13653, 631, 13650, 2559, 2634, 15419]
st = STree(input=navigable_parent_ids_list)
[f'st.{fn}' for fn in dir(st) if not fn.startswith('_')]

['st.build', 'st.find', 'st.find_all', 'st.lcs', 'st.root']

In [6]:

st.lcs()

AttributeError: 'STree' object has no attribute 'word_starts'